In [1]:
from importlib import reload
import sys

src_path =   '../src' # change as needed
sys.path.insert(0,src_path)

max_length = 128

In [2]:
import data_generator;reload(data_generator)

train_data, val_data, test_data = data_generator.GetData(max_length)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
100%|██████████| 397080/397080 [02:03<00:00, 3212.44it/s]


        tag  cat  occurences
0    B-MISC    0         800
1     I-LOC    1      328160
2    I-MISC    2      139600
3     I-ORG    3      237760
4     I-PER    4      977800
5         O    5     6661880
6  [nerCLS]    6      397080
7  [nerPAD]    7    39623065
8  [nerSEP]    8      397080
9    [nerX]    9     2063015

                tag  cat  occurences
0  AFRICAN-AMERICAN    0      436788
1          EUROPEAN    1      208467
2         [raceCLS]    2      397080
3         [racePAD]    3    39623065
4         [raceSEP]    4      397080
5           [raceX]    5     9763760

           tag  cat  occurences
0       FEMALE    0      367299
1         MALE    1      277956
2  [genderCLS]    2      397080
3  [genderPAD]    3    39623065
4  [genderSEP]    4      397080
5    [genderX]    5     9763760



In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import model_utils; reload(model_utils)
    
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = model_utils.NER(max_length)
    
model.generate(bert_train_layers=4)

In [ ]:
model.fit(
    sess,
    train_data,
    val_data,
    epochs=8,
    batch_size=32,
    debias=True
)

In [ ]:
bias = model.getBiasedPValues(test_data, num_iterations=1000)

In [ ]:
bias

In [ ]:
sess.close()